# Évaluer sur l'ensemble de test (10 %) 

In [2]:
from comet_ml import API
import joblib

In [26]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

### Loading models

In [4]:
load_dotenv(r".env")
comet_api_key = os.environ.get('COMET_API_KEY')

In [11]:
api = API(api_key=comet_api_key)

#get the Model object
model = api.get_model(workspace="ift6758-a02", model_name='best-xgboost-model')

#Download a Registry Model:
model.download("2.0.0", expand=True)

In [16]:
loaded_model = joblib.load('None/best_XGBoost_selected_features_xgb.json')

### Loading data

In [17]:
data_path = os.environ.get('DATA_DIR')

In [39]:
test_data_path = data_path+"\\nhl_data_2020_previous_v2_0.csv"
test_data=pd.read_csv(test_data_path)

In [40]:
test_data.sample(3)

,period,periodTimeInSeconds,isGoal,typeDeTir,x,y,distanceToNet,relativeAngleToNet,previousEventTypeId,previousX,previousY,distanceFromPrevious,timeDiff,rebond,angleChange,vitesse,season
43818,1,1138,0,Wrist Shot,-69.0,-21.0,29.698485,-45.000000,BLOCKED_SHOT,-68.0,-9.0,12.04,7,False,0.000000,1.72000,2020
36110,2,489,1,Snap Shot,82.0,6.0,10.000000,1.997880,SHOT,75.0,-14.0,21.19,0,True,6.847724,inf,2020
43750,1,996,0,Wrist Shot,75.0,30.0,33.541020,10.304846,HIT,97.0,-15.0,50.09,8,False,0.000000,6.26125,2020


In [41]:
test_data.dropna(inplace=True)
test_data = test_data.replace([np.inf, -np.inf], np.nan).dropna()

In [42]:
dummy_object = pd.get_dummies(test_data[['typeDeTir', 'previousEventTypeId']])
data = test_data.merge(dummy_object, left_index=True, right_index=True)
test_data_fin = data.drop(labels = ['typeDeTir', 'previousEventTypeId'], axis = 1)

In [46]:
selected_features = ['period', 'periodTimeInSeconds', 'y', 'distanceToNet', 'timeDiff', 'rebond', 'vitesse', 'typeDeTir_Backhand', 'typeDeTir_Deflected', 'typeDeTir_Slap Shot', 'typeDeTir_Snap Shot', 'typeDeTir_Tip-In', 'typeDeTir_Wrap-around', 'typeDeTir_Wrist Shot', 'previousEventTypeId_FACEOFF', 'previousEventTypeId_GIVEAWAY', 'previousEventTypeId_HIT', 'previousEventTypeId_MISSED_SHOT']

In [47]:
X_test = test_data_fin.loc[:,selected_features]
y_test = test_data_fin['isGoal']

In [48]:
y_pred = loaded_model.predict(X_test)

ValueError: feature_names mismatch: ['distanceToNet', 'relativeAngleToNet'] ['period', 'periodTimeInSeconds', 'y', 'distanceToNet', 'timeDiff', 'rebond', 'vitesse', 'typeDeTir_Backhand', 'typeDeTir_Deflected', 'typeDeTir_Slap Shot', 'typeDeTir_Snap Shot', 'typeDeTir_Tip-In', 'typeDeTir_Wrap-around', 'typeDeTir_Wrist Shot', 'previousEventTypeId_FACEOFF', 'previousEventTypeId_GIVEAWAY', 'previousEventTypeId_HIT', 'previousEventTypeId_MISSED_SHOT']
expected relativeAngleToNet in input data
training data did not have the following fields: typeDeTir_Wrist Shot, typeDeTir_Wrap-around, previousEventTypeId_FACEOFF, previousEventTypeId_GIVEAWAY, typeDeTir_Snap Shot, previousEventTypeId_MISSED_SHOT, vitesse, rebond, typeDeTir_Slap Shot, timeDiff, period, y, typeDeTir_Deflected, typeDeTir_Tip-In, periodTimeInSeconds, typeDeTir_Backhand, previousEventTypeId_HIT